# Split the data into train and test data sets

The test data set should have enough data to allow for a good check of the model, with 1 million rows this will probably amount to a 95/5 split

should check metrics with unique counts of values after the split to validate that the test dataset will be good for evaluation purposes.